In [2]:
!pip install fastmcp

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached platformdirs-4.5.0-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.8 MB/s  0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached platformdirs-4.5.0-py3-none-any.whl (18 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.1
    Uninstalling python-dotenv-1.0.1:
      Successfully uninstalled python-dotenv-1.0.1
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.10.0
    Uninstalling platformdirs-3.10.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/22 [platformdirs]
      Successfully uninstalled platformdirs-3.10.0━━━━━━━━━━━━  3/22 [platformdirs]
  Attempting uninstall: cachetools━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/22 [platformdirs]
    Found existing installation: cachetools 5.3.3━━━━━━━━━━━━━  3/22 [platformdirs]
    Uninstalling cachetools-5.3.3:━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/22 [platfor

In [9]:
import json
import os
import snowflake.connector
from snowflake.connector import DictCursor
from dotenv import load_dotenv
import requests
from fastmcp import FastMCP

print("="*70)
print("LAB 8: MODEL CONTEXT PROTOCOL WITH FASTMCP")
print("="*70)

LAB 8: MODEL CONTEXT PROTOCOL WITH FASTMCP


In [10]:
load_dotenv()

SNOWFLAKE_CONFIG = {
    'user': os.getenv('SNOWFLAKE_USER'),
    'password': os.getenv('SNOWFLAKE_PASSWORD'),
    'account': os.getenv('SNOWFLAKE_ACCOUNT'),
    'warehouse': os.getenv('SNOWFLAKE_WAREHOUSE'),
    'database': os.getenv('SNOWFLAKE_DATABASE'),
    'schema': os.getenv('SNOWFLAKE_SCHEMA'),
    'role': os.getenv('SNOWFLAKE_ROLE', 'PUBLIC')
}

print("✅ Configuration loaded")

✅ Configuration loaded


In [15]:
print("\n" + "="*70)
print("PART 1: DATABASE MCP SERVER")
print("="*70)

# Create MCP server for database
database_mcp = FastMCP("Database Server")

@database_mcp.tool()
def query_database(sql: str) -> dict:
    """
    Execute a SQL query on Snowflake database
    
    Args:
        sql: SQL query to execute
    
    Returns:
        Dictionary with query results
    """
    print(f"\n[DB TOOL] query_database")
    print(f"[DB TOOL] SQL: {sql[:80]}...")
    
    try:
        conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
        cursor = conn.cursor(DictCursor)
        cursor.execute(sql)
        results = cursor.fetchall()
        cursor.close()
        conn.close()
        
        print(f"[DB TOOL] ✅ {len(results)} rows")
        return {"success": True, "rows": results, "row_count": len(results)}
    except Exception as e:
        print(f"[DB TOOL] ❌ {str(e)}")
        return {"success": False, "error": str(e)}

@database_mcp.tool()
def list_tables() -> dict:
    """
    List all tables in the Snowflake database
    
    Returns:
        Dictionary with list of table names
    """
    print(f"\n[DB TOOL] list_tables")
    
    try:
        conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
        cursor = conn.cursor()
        cursor.execute("SHOW TABLES")
        tables = [row[1] for row in cursor.fetchall()]
        cursor.close()
        conn.close()
        
        print(f"[DB TOOL] ✅ {len(tables)} tables")
        return {"success": True, "tables": tables}
    except Exception as e:
        print(f"[DB TOOL] ❌ {str(e)}")
        return {"success": False, "error": str(e)}

@database_mcp.tool()
def search_reviews_rag(query: str) -> dict:
    """
    Search iPhone reviews using vector similarity (RAG)
    
    Args:
        query: What to search for in reviews
    
    Returns:
        Dictionary with retrieved reviews
    """
    print(f"\n[DB TOOL] search_reviews_rag")
    print(f"[DB TOOL] Query: {query}")
    
    try:
        conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
        cursor = conn.cursor()
        
        escaped = query.replace("'", "''")
        
        print(f"[DB TOOL] Performing RAG...")
        
        rag_query = f"""
        WITH user_query AS (
            SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', '{escaped}') as query_embedding
        ),
        relevant_reviews AS (
            SELECT REVIEWTITLE, REVIEWDESCRIPTION, RATINGSCORE,
                   VECTOR_COSINE_SIMILARITY(REVIEW_EMBEDDINGS::VECTOR(FLOAT, 1024), uq.query_embedding) AS similarity_score
            FROM LAB_DB.PUBLIC.IPHONE_TABLE it
            CROSS JOIN user_query uq
            WHERE REVIEW_EMBEDDINGS IS NOT NULL
              AND VECTOR_COSINE_SIMILARITY(REVIEW_EMBEDDINGS::VECTOR(FLOAT, 1024), uq.query_embedding) > 0.3
            ORDER BY similarity_score DESC LIMIT 5
        )
        SELECT LISTAGG(REVIEWTITLE || ' (Rating: ' || RATINGSCORE || ') - ' || REVIEWDESCRIPTION, ' | ') 
        WITHIN GROUP (ORDER BY similarity_score DESC) as retrieved_reviews
        FROM relevant_reviews
        """
        
        cursor.execute(rag_query)
        result = cursor.fetchone()
        reviews = result[0] if result and result[0] else "No reviews found"
        
        cursor.close()
        conn.close()
        
        print(f"[DB TOOL] ✅ Retrieved reviews")
        return {"success": True, "retrieved_reviews": reviews}
    except Exception as e:
        print(f"[DB TOOL] ❌ {str(e)}")
        return {"success": False, "error": str(e)}

print("✅ Database MCP Server created with 3 tools")


PART 1: DATABASE MCP SERVER
✅ Database MCP Server created with 3 tools


In [16]:
print("\n" + "="*70)
print("PART 2: NEWS MCP SERVER")
print("="*70)

news_mcp = FastMCP("News Server")

@news_mcp.tool()
def get_latest_news(topic: str) -> dict:
    """
    Get latest news articles about iPhones
    
    Args:
        topic: What news to search for
    
    Returns:
        Dictionary with news articles
    """
    print(f"\n[NEWS TOOL] get_latest_news: {topic}")
    
    api_key = os.getenv('SERPAPI_API_KEY')
    if not api_key:
        return {"success": False, "error": "API key not configured"}
    
    try:
        params = {
            "api_key": api_key,
            "engine": "google",
            "q": f"{topic} iPhone news",
            "tbm": "nws",
            "num": 5
        }
        
        print(f"[NEWS TOOL] Calling SerpAPI...")
        response = requests.get("https://serpapi.com/search", params=params)
        
        if response.status_code != 200:
            return {"success": False, "error": f"HTTP {response.status_code}"}
        
        data = response.json()
        articles = []
        
        if "news_results" in data:
            for article in data["news_results"][:5]:
                articles.append({
                    "title": article.get("title", ""),
                    "snippet": article.get("snippet", ""),
                    "source": article.get("source", "")
                })
        
        print(f"[NEWS TOOL] ✅ {len(articles)} articles")
        return {"success": True, "articles": articles}
    except Exception as e:
        print(f"[NEWS TOOL] ❌ {str(e)}")
        return {"success": False, "error": str(e)}

print("✅ News MCP Server created with 1 tool")



PART 2: NEWS MCP SERVER
✅ News MCP Server created with 1 tool


In [17]:
print("\n" + "="*70)
print("PART 3: LOCATION MCP SERVER")
print("="*70)

location_mcp = FastMCP("Location Server")

@location_mcp.tool()
def find_nearest_store(location: str) -> dict:
    """
    Find nearest Apple Store to a location
    
    Args:
        location: City or address
    
    Returns:
        Dictionary with store information
    """
    print(f"\n[LOCATION TOOL] find_nearest_store: {location}")
    
    api_key = os.getenv('GOOGLE_MAPS_API_KEY')
    if not api_key:
        return {"success": False, "error": "API key not configured"}
    
    try:
        url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
        params = {
            "query": f"Apple Store near {location}",
            "key": api_key,
            "type": "store"
        }
        
        print(f"[LOCATION TOOL] Calling Google Maps...")
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            return {"success": False, "error": f"HTTP {response.status_code}"}
        
        data = response.json()
        stores = []
        
        if "results" in data:
            for store in data["results"][:2]:
                stores.append({
                    "name": store.get("name", ""),
                    "address": store.get("formatted_address", ""),
                    "rating": store.get("rating", "N/A")
                })
        
        print(f"[LOCATION TOOL] ✅ {len(stores)} stores")
        return {"success": True, "stores": stores}
    except Exception as e:
        print(f"[LOCATION TOOL] ❌ {str(e)}")
        return {"success": False, "error": str(e)}

print("✅ Location MCP Server created with 1 tool")


PART 3: LOCATION MCP SERVER
✅ Location MCP Server created with 1 tool


In [18]:
print("\n" + "="*70)
print("PART 4: MCP CLIENT")
print("="*70)

class MCPClient:
    """
    MCP Client for multiple FastMCP servers
    Uses FastMCP's built-in methods - NO hardcoded dispatch
    """
    
    def __init__(self):
        self.servers = {}
        print("[CLIENT] Initialized")
    
    def register_server(self, name: str, server: FastMCP):
        """Register a FastMCP server"""
        self.servers[name] = server
        print(f"[CLIENT] Registered: {name}")
    
    def discover_all_tools(self) -> list:
        """
        Discover tools from all servers
        Note: This is a simplified approach for Jupyter notebook
        In production, client would connect via STDIO/HTTP
        """
        print("\n" + "="*70)
        print("[CLIENT] 🔍 DISCOVERY")
        print("="*70)
        
        all_tools = []
        
        for server_name, server in self.servers.items():
            print(f"\n[CLIENT] Discovering: {server_name}")
            
            # For notebook demo: manually list the decorated tools
            # In production: client would call tools/list via JSON-RPC
            if server_name == "database-server":
                server_tools = [
                    {"name": "query_database", "description": "Execute SQL query", "parameters": {"sql": "string"}},
                    {"name": "list_tables", "description": "List all tables", "parameters": {}},
                    {"name": "search_reviews_rag", "description": "Search reviews with RAG", "parameters": {"query": "string"}}
                ]
            elif server_name == "news-server":
                server_tools = [
                    {"name": "get_latest_news", "description": "Get latest iPhone news", "parameters": {"topic": "string"}}
                ]
            elif server_name == "location-server":
                server_tools = [
                    {"name": "find_nearest_store", "description": "Find Apple Stores", "parameters": {"location": "string"}}
                ]
            else:
                server_tools = []
            
            for tool in server_tools:
                tool['server'] = server_name
                all_tools.append(tool)
                print(f"[CLIENT]    - {tool['name']}: {tool['description']}")
        
        print(f"\n[CLIENT] ✅ {len(all_tools)} tools from {len(self.servers)} servers")
        print("="*70)
        
        return all_tools
    
    def call_tool(self, server_name: str, tool_name: str, arguments: dict) -> dict:
        """
        Call a tool on a server
        Note: Simplified for Jupyter - directly calls the Python functions
        In production: would send JSON-RPC request
        """
        print("\n" + "="*70)
        print("[CLIENT] 📤 TOOL CALL")
        print("="*70)
        print(f"[CLIENT] {server_name}.{tool_name}")
        print(f"[CLIENT] Args: {arguments}")
        
        if server_name not in self.servers:
            return {"error": f"Server {server_name} not found"}
        
        # For notebook: directly call the tool functions
        # In production: would use JSON-RPC protocol
        print(f"[CLIENT] Executing...")
        try:
            if server_name == "database-server":
                if tool_name == "query_database":
                    result = query_database(**arguments)
                elif tool_name == "list_tables":
                    result = list_tables()
                elif tool_name == "search_reviews_rag":
                    result = search_reviews_rag(**arguments)
                else:
                    result = {"error": "Tool not found"}
            
            elif server_name == "news-server":
                if tool_name == "get_latest_news":
                    result = get_latest_news(**arguments)
                else:
                    result = {"error": "Tool not found"}
            
            elif server_name == "location-server":
                if tool_name == "find_nearest_store":
                    result = find_nearest_store(**arguments)
                else:
                    result = {"error": "Tool not found"}
            
            else:
                result = {"error": "Server not found"}
            
            print(f"[CLIENT] ✅ Success")
            return result
        except Exception as e:
            print(f"[CLIENT] ❌ {str(e)}")
            return {"error": str(e)}

# Create client and register servers
client = MCPClient()
client.register_server("database-server", database_mcp)
client.register_server("news-server", news_mcp)
client.register_server("location-server", location_mcp)

# Discover tools
tools = client.discover_all_tools()

print(f"\n✅ Client ready with {len(tools)} tools")


PART 4: MCP CLIENT
[CLIENT] Initialized
[CLIENT] Registered: database-server
[CLIENT] Registered: news-server
[CLIENT] Registered: location-server

[CLIENT] 🔍 DISCOVERY

[CLIENT] Discovering: database-server
[CLIENT]    - query_database: Execute SQL query
[CLIENT]    - list_tables: List all tables
[CLIENT]    - search_reviews_rag: Search reviews with RAG

[CLIENT] Discovering: news-server
[CLIENT]    - get_latest_news: Get latest iPhone news

[CLIENT] Discovering: location-server
[CLIENT]    - find_nearest_store: Find Apple Stores

[CLIENT] ✅ 5 tools from 3 servers

✅ Client ready with 5 tools


In [30]:
# Re-run MCPClient class definition
class MCPClient:
    def __init__(self):
        self.servers = {}
        self.tool_functions = {}
        print("[CLIENT] Initialized")
    
    def register_server(self, name: str, server: FastMCP, tools_map: dict):
        self.servers[name] = server
        
        for tool_name, func in tools_map.items():
            key = f"{name}.{tool_name}"
            
            # Extract actual callable from FunctionTool wrapper
            actual_func = func
            if hasattr(func, 'fn'):
                actual_func = func.fn
            elif hasattr(func, '__wrapped__'):
                actual_func = func.__wrapped__
            
            self.tool_functions[key] = {
                'function': actual_func,
                'server': name,
                'name': tool_name,
                'description': actual_func.__doc__ or ""
            }
        
        print(f"[CLIENT] Registered: {name} with {len(tools_map)} tools")
    
    def discover_all_tools(self) -> list:
        print("\n" + "="*70)
        print("[CLIENT] 🔍 DISCOVERY")
        print("="*70)
        
        all_tools = []
        current_server = None
        
        for key, tool_info in self.tool_functions.items():
            tool_data = {
                'server': tool_info['server'],
                'name': tool_info['name'],
                'description': tool_info['description'].split('\n')[0].strip() if tool_info['description'] else "",
                'parameters': {}
            }
            all_tools.append(tool_data)
            
            if current_server != tool_info['server']:
                print(f"\n[CLIENT] {tool_info['server']}:")
                current_server = tool_info['server']
            
            print(f"[CLIENT]    - {tool_data['name']}: {tool_data['description'][:60]}")
        
        print(f"\n[CLIENT] ✅ {len(all_tools)} tools from {len(self.servers)} servers")
        print("="*70)
        
        return all_tools
    
    def call_tool(self, server_name: str, tool_name: str, arguments: dict) -> dict:
        print("\n" + "="*70)
        print("[CLIENT] 📤 TOOL CALL")
        print("="*70)
        print(f"[CLIENT] {server_name}.{tool_name}")
        print(f"[CLIENT] Args: {arguments}")
        
        key = f"{server_name}.{tool_name}"
        
        if key not in self.tool_functions:
            print(f"[CLIENT] ❌ Tool not found")
            return {"error": f"Tool {key} not found"}
        
        func = self.tool_functions[key]['function']
        
        print(f"[CLIENT] Executing...")
        try:
            result = func(**arguments)
            print(f"[CLIENT] ✅ Success")
            return result
        except Exception as e:
            print(f"[CLIENT] ❌ {str(e)}")
            return {"error": str(e)}

# Register servers
client = MCPClient()

client.register_server("database-server", database_mcp, {
    "query_database": query_database,
    "list_tables": list_tables,
    "search_reviews_rag": search_reviews_rag
})

client.register_server("news-server", news_mcp, {
    "get_latest_news": get_latest_news
})

client.register_server("location-server", location_mcp, {
    "find_nearest_store": find_nearest_store
})

tools = client.discover_all_tools()

[CLIENT] Initialized
[CLIENT] Registered: database-server with 3 tools
[CLIENT] Registered: news-server with 1 tools
[CLIENT] Registered: location-server with 1 tools

[CLIENT] 🔍 DISCOVERY

[CLIENT] database-server:
[CLIENT]    - query_database: 
[CLIENT]    - list_tables: 
[CLIENT]    - search_reviews_rag: 

[CLIENT] news-server:
[CLIENT]    - get_latest_news: 

[CLIENT] location-server:
[CLIENT]    - find_nearest_store: 

[CLIENT] ✅ 5 tools from 3 servers


In [27]:
print("\n" + "="*70)
print("PART 5: LLM BRAIN")
print("="*70)

class CortexBrain:
    """Snowflake Cortex LLM for routing decisions"""
    
    def __init__(self, config: dict):
        self.config = config
        print("[BRAIN] Initialized with Snowflake Cortex")
    
    def decide_tool(self, user_query: str, available_tools: list) -> dict:
        """Use Cortex to decide which tool"""
        print("\n" + "="*70)
        print("[BRAIN] 🧠 DECISION")
        print("="*70)
        print(f"[BRAIN] Query: '{user_query}'")
        print(f"[BRAIN] {len(available_tools)} tools available")
        
        tools_text = "\n\n".join([
            f"Server: {t['server']}\n"
            f"Tool: {t['name']}\n"
            f"Description: {t['description']}\n"
            f"Parameters: {json.dumps(t['parameters'])}"
            for t in available_tools
        ])
        
        prompt = f"""
You are a routing agent.

Available tools:
{tools_text}

User: {user_query}

Choose best tool. JSON only:
{{"server": "name", "tool_name": "name", "arguments": {{}}, "reasoning": "why"}}
"""
        
        try:
            conn = snowflake.connector.connect(**self.config)
            cursor = conn.cursor()
            
            query = f"SELECT SNOWFLAKE.CORTEX.COMPLETE('mistral-large2', '{prompt.replace(chr(39), chr(39)*2)}') as decision"
            
            print(f"[BRAIN] Calling Cortex...")
            cursor.execute(query)
            result = cursor.fetchone()
            decision_text = result[0] if result else "{}"
            
            cursor.close()
            conn.close()
            
            # Parse
            decision_text = decision_text.strip()
            if "```json" in decision_text:
                decision_text = decision_text.split("```json")[1].split("```")[0]
            elif "```" in decision_text:
                decision_text = decision_text.split("```")[1].split("```")[0]
            
            decision = json.loads(decision_text.strip())
            
            print(f"[BRAIN] 🎯 {decision.get('server')}.{decision['tool_name']}")
            print(f"[BRAIN] {decision.get('reasoning', '')[:60]}")
            
            return decision
        except Exception as e:
            print(f"[BRAIN] ❌ {str(e)}")
            return {
                "server": available_tools[0]['server'] if available_tools else "database-server",
                "tool_name": available_tools[0]['name'] if available_tools else "list_tables",
                "arguments": {},
                "reasoning": "error fallback"
            }

brain = CortexBrain(SNOWFLAKE_CONFIG)


PART 5: LLM BRAIN
[BRAIN] Initialized with Snowflake Cortex


In [31]:
print("\n" + "="*70)
print("PART 6: MCP AGENT")
print("="*70)

class MCPAgent:
    """Agent orchestrating Brain + Client + Servers"""
    
    def __init__(self, brain: CortexBrain, client: MCPClient):
        self.brain = brain
        self.client = client
        print("[AGENT] Ready")
    
    def process_query(self, query: str):
        """Process query through MCP"""
        print("\n" + "="*70)
        print("🚀 PROCESSING")
        print("="*70)
        print(f"Query: '{query}'")
        
        # Discover current tools
        tools = self.client.discover_all_tools()
        
        # Brain decides
        decision = self.brain.decide_tool(query, tools)
        
        # Client executes
        result = self.client.call_tool(
            decision['server'],
            decision['tool_name'],
            decision.get('arguments', {})
        )
        
        # Display
        print("\n📊 RESULT:")
        if result.get('success'):
            print(json.dumps(result, indent=2, default=str)[:400])
        else:
            print(f"❌ {result.get('error')}")
        
        return result

agent = MCPAgent(brain, client)


PART 6: MCP AGENT
[AGENT] Ready


In [32]:
agent.process_query("Show me all tables")


🚀 PROCESSING
Query: 'Show me all tables'

[CLIENT] 🔍 DISCOVERY

[CLIENT] database-server:
[CLIENT]    - query_database: 
[CLIENT]    - list_tables: 
[CLIENT]    - search_reviews_rag: 

[CLIENT] news-server:
[CLIENT]    - get_latest_news: 

[CLIENT] location-server:
[CLIENT]    - find_nearest_store: 

[CLIENT] ✅ 5 tools from 3 servers

[BRAIN] 🧠 DECISION
[BRAIN] Query: 'Show me all tables'
[BRAIN] 5 tools available
[BRAIN] Calling Cortex...
[BRAIN] 🎯 database-server.list_tables
[BRAIN] The user wants to see all tables, so the 'list_tables' tool 

[CLIENT] 📤 TOOL CALL
[CLIENT] database-server.list_tables
[CLIENT] Args: {}
[CLIENT] Executing...

[DB TOOL] list_tables
[DB TOOL] ✅ 7 tables
[CLIENT] ✅ Success

📊 RESULT:
{
  "success": true,
  "tables": [
    "CHAT_MESSAGES",
    "CHAT_SESSIONS",
    "CHECKPOINTS",
    "EPSS_DAILY",
    "IPHONE_TABLE",
    "MEMORY_STORE",
    "SHORT_TERM_MEMORY"
  ]
}


{'success': True,
 'tables': ['CHAT_MESSAGES',
  'CHAT_SESSIONS',
  'CHECKPOINTS',
  'EPSS_DAILY',
  'IPHONE_TABLE',
  'MEMORY_STORE',
  'SHORT_TERM_MEMORY']}